<a href="https://colab.research.google.com/github/ekene0013/Projects-on-Derivative-Pricing-at-WQU/blob/main/GWP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Scenario**

**As you have upgraded your option pricing skill set, you can demonstrate those skills by pricing the same options as before. The group is now asked to assess the causes of volatility by 2 properties: stochastic volatility and jumps in the underlying. Speciﬁcally, you will use the Heston Model and Merton Model instead of the Black-Scholes Model.**


**Task: This assignment is based on the same questions as in GWP1 and GWP2. To maintain the same numbering, questions begin from #5. Remember to use the following general parameters:**
S=80; r = 5.5%; σ = 35%; T = 3 months


**Also, remember that you will be employing Monte-Carlo method, so make sure you run a suﬃcient number of simulations and consider enough time steps (for you to determine).**

**Step 1:**

**Team Member A:  Stochastic Volatility Modeler:**

**For the Heston model, you can use the following parameters:**

- $\nu_0$ = 3.2%
- $k_v$ = 1.85
- $\theta_v$ = 0.045









In [ ]:
# @title


**5(a) Using the Heston Model and Monte-Carlo simulation, price an ATM European call and put, using a correlation value of -0.30**

To price an ATM (At-The-Money) European call and put option using the Heston model and Monte Carlo simulation with a correlation value of -0.30, you would initialize model and simulation parameters, generate random numbers for both asset and volatility processes, create a covariance matrix to account for correlation, simulate volatility paths, then simulate asset price paths while incorporating the volatility paths. Next, calculate option payoffs for call and put options at maturity, discount these payoffs to present value using the risk-free rate, and average the results across all simulations. The resulting average represents the estimated prices of the ATM European call and put options in the Heston model with Monte Carlo simulation (Schumacher, 2020).

**5(b) An overview on the calculated estimates**

Following the computation below, the call is valued at \$2.83, while the put is worth \$2.81. European call and put options are often priced closely to each other, with put options slightly cheaper due to the opportunity cost of holding cash instead of the underlying asset.

In [ ]:
# @title
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=np.float)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S

def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand
np.random.seed(42)

v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.30

S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 50  # Number of time steps in a year
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 10000  # Number of simulations
dt = T / M  # Length of time step

# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)
np.random.seed(42)

# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

def heston_call_mc(S, K, r, T, t, type_of):
    if type_of == "C":
        payoff = np.maximum(0, S[-1, :] - K)
    else:
        payoff = np.maximum(0, K - S[-1, :])

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

<ipython-input-1-6de0f0dfe501>:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  covariance_matrix = np.zeros((2, 2), dtype=np.float)
<ipython-input-1-6de0f0dfe501>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v = np.zeros((M + 1, Ite), dtype=np.float)


In [ ]:
# @title
print("European Call Price under Heston: ", heston_call_mc(S, 80, 0.055, 3/12, 0, "C"))
print("European Put Price under Heston: ", heston_call_mc(S, 80, 0.055, 3/12, 0, "P"))

European Call Price under Heston:  2.889397739971047
European Put Price under Heston:  2.8079276452360995


In [ ]:
# @title
from tabulate import tabulate

# Prepare your data as a list of lists
table_data = [
    ["5", "Call", "European", "Heston Model in combination with Monte-Carlo simulation and a correlation value of -0.30", "$2.83"],
    ["5", "Put", "European", "Heston Model in combination with Monte-Carlo simulation and a correlation value of -0.30", "$2.81"]
]

# Define the title for the table
table_title = "Summary of Results"

# Define the table headers
headers = ["Q #s", "Type", "Exer", "GWP 3 Method", "Price"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

Summary of Results

+--------+--------+----------+------------------------------------------------------------------------------------------+---------+
|   Q #s | Type   | Exer     | GWP 3 Method                                                                             | Price   |
+========+========+==========+==========================================================================================+=========+
|      5 | Call   | European | Heston Model in combination with Monte-Carlo simulation and a correlation value of -0.30 | $2.83   |
+--------+--------+----------+------------------------------------------------------------------------------------------+---------+
|      5 | Put    | European | Heston Model in combination with Monte-Carlo simulation and a correlation value of -0.30 | $2.81   |
+--------+--------+----------+------------------------------------------------------------------------------------------+---------+


**6. Using the Heston Model, price an ATM European call and put, using a correlation value of -0.70**

When the correlation value changes in the Heston model from -0.30 to -0.70, there is a significant change in the pricing of an ATM European call and put option, with the call valued at \$2.01 and the put at \$3.49. This is primarily because of the effect of correlation on option pricing dynamics. When the asset price rises, there is a reduced volatility due to the larger negative correlation between asset returns and volatility returns at -0.70, which lowers the cost of the call option. In contrast, the put option benefits from the negative correlation, increasing its value by protecting against increased downside risk and driving up the price of the put option. This modification illustrates the role that correlation dynamics play in determining the sensitivity of option pricing as well as market sentiment and model calibration in option pricing (Schumacher, 2020).

In [ ]:
# @title
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=np.float)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S

def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand
np.random.seed(42)

v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.70

S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 50  # Number of time steps in a year
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 10000  # Number of simulations
dt = T / M  # Length of time step

# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)
np.random.seed(42)


# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

def heston_call_mc(S, K, r, T, t, type_of):
    if type_of == "C":
        payoff = np.maximum(0, S[-1, :] - K)
    else:
        payoff = np.maximum(0, K - S[-1, :])

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

<ipython-input-4-388ef42152f8>:55: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  covariance_matrix = np.zeros((2, 2), dtype=np.float)
<ipython-input-4-388ef42152f8>:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v = np.zeros((M + 1, Ite), dtype=np.float)


In [ ]:
# @title
print("European Call Price under Heston: ", heston_call_mc(S, 80, 0.055, 3/12, 0, "C"))
print("European Put Price under Heston: ", heston_call_mc(S, 80, 0.055, 3/12, 0, "P"))

European Call Price under Heston:  2.100269052154384
European Put Price under Heston:  3.4294332097648033


In [ ]:
# @title
from tabulate import tabulate

# Prepare your data as a list of lists
table_data = [
    ["6", "Call", "European", "Heston Model with a correlation value of -0.70", "$2.01"],
    ["6", "Put", "European", "Heston Model with a correlation value of -0.70", "$3.49"]
]

# Define the title for the table
table_title = "Summary of Results"

# Define the table headers
headers = ["Q #s", "Type", "Exer", "GWP 3 Method", "Price"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

Summary of Results

+--------+--------+----------+------------------------------------------------+---------+
|   Q #s | Type   | Exer     | GWP 3 Method                                   | Price   |
+========+========+==========+================================================+=========+
|      6 | Call   | European | Heston Model with a correlation value of -0.70 | $2.01   |
+--------+--------+----------+------------------------------------------------+---------+
|      6 | Put    | European | Heston Model with a correlation value of -0.70 | $3.49   |
+--------+--------+----------+------------------------------------------------+---------+


**7(a) Calculate delta and gamma for each of the options in Questions 5. (Hint: You can numerically approximate this by forcing a change in the variable of interest –i.e., underlying stock price and delta change—and recalculate the option price).**

In the context of the Heston Model and Monte Carlo simulation for option pricing, "delta" and "gamma" are Greek letters used to represent two important risk sensitivities of options.

Delta represents the sensitivity of an option's price to changes in the underlying asset's price. In the context of the Heston Model and Monte Carlo simulation, delta can be calculated by simulating a small change in the asset price, reevaluating the option's price in each simulation, and calculating the average change. It can take on values between -1 and 1 for call options and between -1 and 0 for put options. In the context of the Heston Model and Monte Carlo simulation with a rho of -0.30, we find delta to be 0.56 for the call option and 0.70 for the put option (Schumacher, 2020).

On the other hand, Gamma measures the rate of change of an option's delta with respect to changes in the underlying asset's price. In the Heston Model and Monte Carlo simulation, gamma can be calculated by simulating small changes in the asset price and corresponding changes in delta, then calculating the second derivative of the option price with respect to the asset price (Γ = ∂²V/∂S²). This is why Gamma can either ne positive or negative. It can especially be negative typically for long options positions. In the context of the Heston Model and Monte Carlo simulation with a rho of -0.30, we find delta to be -0.42 for the call option and -0.53 for the put option (Schumacher, 2020).

In [ ]:
# @title
import numpy as np

# Heston Model Parameters
v0 = 0.032
kappa_v = 1.85
theta_v = 0.045
sigma_v = 0.35
rho = -0.30

# Option Parameters
S0 = 80
K_call = 80  # ATM Call Strike Price
K_put = 80   # ATM Put Strike Price
r = 0.055  # Risk-free rate
T = 3/12  # Time to maturity (in years)
M0 = 50   # Number of time steps in a year
M = int(M0 * T)  # Total time steps
Ite = 10000  # Number of simulations
dt = T / M  # Length of time step

# Small perturbation for delta calculation
delta_S = 0.01

# Generate random numbers
rand = np.random.standard_normal((2, M + 1, Ite))
np.random.seed(42)

# Covariance Matrix
covariance_matrix = np.array([[1.0, rho], [rho, 1.0]])
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Function to simulate volatility paths
def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M
    v = np.zeros((M + 1, Ite), dtype=np.float)
    v[0] = v0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1] + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

# Function to simulate stock price paths
def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)
    return S

# Calculate option prices at the original stock price
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)
S = Heston_paths(S0, r, V, 0, cho_matrix)

call_price_original = np.mean(np.maximum(0, S[-1] - K_call) * np.exp(-r * T))
put_price_original = np.mean(np.maximum(0, K_put - S[-1]) * np.exp(-r * T))

# Calculate option prices with a slightly increased stock price
S_perturbed = S0 * (1 + delta_S)
V_perturbed = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)
S_perturbed = Heston_paths(S_perturbed, r, V_perturbed, 0, cho_matrix)

call_price_perturbed = np.mean(np.maximum(0, S_perturbed[-1] - K_call) * np.exp(-r * T))
put_price_perturbed = np.mean(np.maximum(0, K_put - S_perturbed[-1]) * np.exp(-r * T))

# Calculate delta and gamma for the European call and put options
call_delta = (call_price_perturbed - call_price_original) / (delta_S * S0)
put_delta = (put_price_perturbed - put_price_original) / (delta_S * S0)

# Calculate gamma using finite differences
call_gamma = ((call_price_perturbed - 2 * call_price_original + call_price_original) / (delta_S**2 * S0**2))
put_gamma = ((put_price_perturbed - 2 * put_price_original + put_price_original) / (delta_S**2 * S0**2))

# Print the results
print("ATM European Call Option Delta:", call_delta)
print("ATM European Call Option Gamma:", call_gamma)
print("ATM European Put Option Delta:", put_delta)
print("ATM European Put Option Gamma:", put_gamma)


ATM European Call Option Delta: 0.5629774694852985
ATM European Call Option Gamma: 0.703721836856623
ATM European Put Option Delta: -0.4243850061659288
ATM European Put Option Gamma: -0.530481257707411


<ipython-input-7-4ca0c757c407>:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v = np.zeros((M + 1, Ite), dtype=np.float)


In [ ]:
# @title
from tabulate import tabulate

# Prepare your data as a list of lists
table_data = [
    ["7", "Call", "European", "Heston Model in combination with Monte-Carlo simulation and a correlation value of -0.30", "$2.83", "0.56", "0.70"],
    ["7", "Put", "European", "Heston Model in combination with Monte-Carlo simulation and a correlation value of -0.30", "$2.81", "-0.42", "-0.53"]
]

# Define the title for the table
table_title = "Summary of Results for Delta and Gamma of each options in Question 5"

# Define the table headers
headers = ["Q #s", "Type", "Exer", "GWP 3 Method", "Price", "Delta", "Gamma"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

Summary of Results for Delta and Gamma of each options in Question 5

+--------+--------+----------+------------------------------------------------------------------------------------------+---------+---------+---------+
|   Q #s | Type   | Exer     | GWP 3 Method                                                                             | Price   |   Delta |   Gamma |
+========+========+==========+==========================================================================================+=========+=========+=========+
|      7 | Call   | European | Heston Model in combination with Monte-Carlo simulation and a correlation value of -0.30 | $2.83   |    0.56 |    0.7  |
+--------+--------+----------+------------------------------------------------------------------------------------------+---------+---------+---------+
|      7 | Put    | European | Heston Model in combination with Monte-Carlo simulation and a correlation value of -0.30 | $2.81   |   -0.42 |   -0.53 |
+--------+--------

**7(b) Calculate delta and gamma for each of the options in Questions 6. (Hint: You can numerically approximate this by forcing a change in the variable of interest –i.e., underlying stock price and delta change—and recalculate the option price).**

Changes in correlation, especially a stronger negative correlation as seen in the shift from -0.30 to -0.70, can significantly impact the relationship between the asset price, volatility, delta, and gamma in the Heston model. This can lead to shifts in the sensitivity of options to price and volatility changes, influencing their delta and gamma values. These changes reflect the evolving dynamics of option pricing in response to different market conditions and model assumptions (Schumacher, 2020).

In our case, we find the delta for the call and put options to be 0.51 and 0.63 respectively. On the other hand, we find the gamma for the call and put options to be -0.46 and -0.58 respectively.

In [ ]:
# @title
import numpy as np

# Heston Model Parameters
v0 = 0.032
kappa_v = 1.85
theta_v = 0.045
sigma_v = 0.35
rho = -0.70  # Updated correlation value

# Option Parameters
S0 = 80
K_call = 80  # ATM Call Strike Price
K_put = 80   # ATM Put Strike Price
r = 0.055  # Risk-free rate
T = 3/12  # Time to maturity (in years)
M0 = 50   # Number of time steps in a year
M = int(M0 * T)  # Total time steps
Ite = 10000  # Number of simulations
dt = T / M  # Length of time step

# Small perturbation for delta calculation
delta_S = 0.01

# Generate random numbers
rand = np.random.standard_normal((2, M + 1, Ite))
np.random.seed(42)

# Covariance Matrix
covariance_matrix = np.array([[1.0, rho], [rho, 1.0]])
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Function to simulate volatility paths
def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M
    v = np.zeros((M + 1, Ite), dtype=np.float)
    v[0] = v0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1] + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

# Function to simulate stock price paths
def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)
    return S

# Calculate option prices at the original stock price
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)
S = Heston_paths(S0, r, V, 0, cho_matrix)

call_price_original = np.mean(np.maximum(0, S[-1] - K_call) * np.exp(-r * T))
put_price_original = np.mean(np.maximum(0, K_put - S[-1]) * np.exp(-r * T))

# Calculate option prices with a slightly increased stock price
S_perturbed = S0 * (1 + delta_S)
V_perturbed = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)
S_perturbed = Heston_paths(S_perturbed, r, V_perturbed, 0, cho_matrix)

call_price_perturbed = np.mean(np.maximum(0, S_perturbed[-1] - K_call) * np.exp(-r * T))
put_price_perturbed = np.mean(np.maximum(0, K_put - S_perturbed[-1]) * np.exp(-r * T))

# Calculate delta and gamma for the European call and put options
call_delta = (call_price_perturbed - call_price_original) / (delta_S * S0)
put_delta = (put_price_perturbed - put_price_original) / (delta_S * S0)

# Calculate gamma using finite differences
call_gamma = ((call_price_perturbed - 2 * call_price_original + call_price_original) / (delta_S**2 * S0**2))
put_gamma = ((put_price_perturbed - 2 * put_price_original + put_price_original) / (delta_S**2 * S0**2))

# Print the results
print("ATM European Call Option Delta (rho -0.70):", call_delta)
print("ATM European Call Option Gamma (rho -0.70):", call_gamma)
print("ATM European Put Option Delta (rho -0.70):", put_delta)
print("ATM European Put Option Gamma (rho -0.70):", put_gamma)


ATM European Call Option Delta (rho -0.70): 0.508109850828728
ATM European Call Option Gamma (rho -0.70): 0.63513731353591
ATM European Put Option Delta (rho -0.70): -0.4616196966681829
ATM European Put Option Gamma (rho -0.70): -0.5770246208352287


<ipython-input-9-55a78b25cba2>:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v = np.zeros((M + 1, Ite), dtype=np.float)


In [ ]:
# @title
from tabulate import tabulate

# Prepare your data as a list of lists
table_data = [
    ["7", "Call", "European", "Heston Model with a correlation value of -0.70", "2.01", "0.51", "0.63"],
    ["7", "Put", "European", "Heston Model with a correlation value of -0.70", "3.49", "-0.46", "-0.58"]
]

# Define the title for the table
table_title = "Summary of Results for Delta and Gamma of each options in Question 6"

# Define the table headers
headers = ["Q #s", "Type", "Exer", "GWP 3 Method", "Price", "Delta", "Gamma"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

Summary of Results for Delta and Gamma of each options in Question 6

+--------+--------+----------+------------------------------------------------+---------+---------+---------+
|   Q #s | Type   | Exer     | GWP 3 Method                                   |   Price |   Delta |   Gamma |
+========+========+==========+================================================+=========+=========+=========+
|      7 | Call   | European | Heston Model with a correlation value of -0.70 |    2.01 |    0.51 |    0.63 |
+--------+--------+----------+------------------------------------------------+---------+---------+---------+
|      7 | Put    | European | Heston Model with a correlation value of -0.70 |    3.49 |   -0.46 |   -0.58 |
+--------+--------+----------+------------------------------------------------+---------+---------+---------+


**Team Member B:  Jump Modeler:**

**For the Merton model, we use the following parameters:**

- $\mu$ = -0.5
- $\delta$ = 0.22

**8. Using the Merton Model, price an ATM European call and put with jump intensity parameter equal to 0.75.**


The Merton model is a jump-diffusion model that is used to price options. The model assumes that the stock price follows a lognormal distribution, but that it can also experience jumps. The jump intensity parameter, λ, controls the frequency of the jumps.

In [ ]:
# @title
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss
np.random.seed(0)



def merton_jump_paths(S0, r, sigma, T, lamb, muJ, deltaJ, n_paths, n_steps):
    dt = T / n_steps
    jump_intensity = lamb * (np.exp(muJ + 0.5 * deltaJ**2) - 1)

    paths = np.zeros((n_steps + 1, n_paths))
    paths[0] = S0

    for t in range(1, n_steps + 1):
        normal_shocks = np.random.normal(0, 1, n_paths)
        jump_shocks = np.random.normal(muJ, deltaJ, n_paths)
        jump_sizes = np.random.poisson(lamb * dt, n_paths)

        paths[t] = paths[t-1] * np.exp((r - 0.5 * sigma**2 - jump_intensity) * dt + sigma * np.sqrt(dt) * normal_shocks + jump_shocks * jump_sizes)

    return paths

def european_call_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps):
    paths = merton_jump_paths(S0, r, sigma, T, lamb, muJ, deltaJ, n_paths, n_steps)
    # Payoff at maturity for European call option
    payoff = np.maximum(paths[-1] - K, 0)
    # Discount the payoff to present value
    return np.mean(payoff) * np.exp(-r * T)

def european_put_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps):
    dt = T / n_steps
    discount = np.exp(-r * T)
    paths = merton_jump_paths(S0, r, sigma, T, lamb, muJ, deltaJ, n_paths, n_steps)

    # Payoff at maturity for a European put
    payoff = np.maximum(K - paths[-1], 0)

    return np.mean(payoff) * discount

S0 = 80
K = 80
r = 0.055
sigma = 0.35
T = 3 / 12
lamb = 0.75
muJ = -0.5
deltaJ = 0.22
n_paths = 100000
n_steps = 100

european_call_price = european_call_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
print(f"European Call Option Price with jump intensity of 0.75: {european_call_price:.2f}")

european_put_price = european_put_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
print(f"European Put Option Price jump intensity of 0.75: {european_put_price:.2f}")





European Call Option Price with jump intensity of 0.75: 8.28
European Put Option Price jump intensity of 0.75: 7.20



**9. Using the Merton Model, price an ATM European call and put with jump intensity parameter equal to 0.25.**


In [ ]:
# @title

S0 = 80
K = 80
r = 0.055
sigma = 0.35
T = 3 / 12
lamb = 0.25
muJ = -0.5
deltaJ = 0.22
n_steps = 100

european_call_price = european_call_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
print(f"European Call Option Price with jump intensity of 0.25: {european_call_price:.2f}")

european_put_price = european_put_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
print(f"European Put Option Price jump intensity of 0.25: {european_put_price:.2f}")



European Call Option Price with jump intensity of 0.25: 6.81
European Put Option Price jump intensity of 0.25: 5.79



**10. Calculate delta and gamma for each of the options in Questions 8 and 9. (Hint: You can use the same trick as in Question 7).**

Delta and gamma are sensitivity measures used in options pricing. They describe how an option's price moves in relation to changes in the underlying asset's price.

In [ ]:
# @title
def compute_delta(S0, deltaS, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps):

    C_S0_plus = european_call_merton(S0 + deltaS, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
    C_S0_minus = european_call_merton(S0 - deltaS, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)

    # Calculate Delta
    Delta = (C_S0_plus - C_S0_minus) / (2 * deltaS)

    return Delta

def compute_gamma(S0, deltaS, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps):

    C_S0 = european_call_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
    C_S0_plus = european_call_merton(S0 + deltaS, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
    C_S0_minus = european_call_merton(S0 - deltaS, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)

    # Calculate Gamma
    Gamma = (C_S0_plus - 2*C_S0 + C_S0_minus) / deltaS**2

    return Gamma

def compute_put_delta_gamma(S0, deltaS, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps):

    P_S0 = european_put_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
    P_S0_plus = european_put_merton(S0 + deltaS, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
    P_S0_minus = european_put_merton(S0 - deltaS, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)

    # Calculate Delta and Gamma
    Delta = (P_S0_plus - P_S0_minus) / (2 * deltaS)
    Gamma = (P_S0_plus - 2*P_S0 + P_S0_minus) / deltaS**2

    return Delta, Gamma


# Parameters
deltaS = 0.001

Gamma = compute_gamma(S0, deltaS, K, r, T, sigma, 0.75, muJ, deltaJ, n_paths, n_steps)
print(f"Gamma of european call with intensity of 0.75:: {Gamma:.4f}")


Delta = compute_delta(S0, deltaS, K, r, T, sigma, 0.75, muJ, deltaJ, n_paths, n_steps)
print(f"Delta of european call with intensity of 0.75: {Delta:.4f}")


Gamma = compute_gamma(S0, deltaS, K, r, T, sigma, 0.25, muJ, deltaJ, n_paths, n_steps)
print(f"Gamma of european call with intensity of 0.25:: {Gamma:.4f}")


Delta = compute_delta(S0, deltaS, K, r, T, sigma, 0.25, muJ, deltaJ, n_paths, n_steps)
print(f"Delta of european call with intensity of 0.25: {Delta:.4f}")

Delta, Gamma = compute_put_delta_gamma(S0, deltaS, K, r, T, sigma, 0.75, muJ, deltaJ, n_paths, n_steps)
print(f"Delta of european put with intensity of 0.75: {Delta:.4f}")
print(f"Gamma of european put with intensity of 0.75:: {Gamma:.4f}")

Delta, Gamma = compute_put_delta_gamma(S0, deltaS, K, r, T, sigma, 0.25, muJ, deltaJ, n_paths, n_steps)
print(f"Delta of european put with intensity of 0.25: {Delta:.4f}")
print(f"Gamma of european put with intensity of 0.25:: {Gamma:.4f}")




Gamma of european call with intensity of 0.75:: 72667.1700
Delta of european call with intensity of 0.75: 21.8533
Gamma of european call with intensity of 0.25:: -1043.0658
Delta of european call with intensity of 0.25: -32.3934
Delta of european put with intensity of 0.75: 11.5227
Gamma of european put with intensity of 0.75:: 11233.2869
Delta of european put with intensity of 0.25: 24.2567
Gamma of european put with intensity of 0.25:: -63593.7835


**Team Member C: Model Validator**

**11. For Questions 5, 6, 8, and 9, use put-call Parity to determine if the prices of the put and call from the Heston Model and Merton Model satisfy put-call parity.**



The put-call parity is $C_{0}= -Ke^{-rT}+S_{0}+P_{0}$,for Question 5 ,
As we can see
 $C_{0}= 2.86,-Ke^{-rT} = -78.91,S_{0} = 80,P_{0}$= 2.83.
So that $C_{0}\neq-Ke^{-rT}+S_{0}+P_{0}$, which means  Put-Call parity is not satisfied under the Heston Model.


The put-call parity is $C_{0}= -Ke^{-rT}+S_{0}+P_{0}$,for Question 6 ,
As we can see
 $C_{0}= 2.09,-Ke^{-rT} = -78.91,S_{0} = 80,P_{0}$= 3.45.
So that $C_{0}\neq-Ke^{-rT}+S_{0}+P_{0}$, which means  Put-Call parity is not satisfied under the Heston Model.

The put-call parity is $C_{0}= -Ke^{-rT}+S_{0}+P_{0}$,for Question 8 ,
As we can see
 $C_{0}= 8.28,-Ke^{-rT} = -78.91,S_{0} = 80,P_{0}$= 7.20.
So that $C_{0}\neq-Ke^{-rT}+S_{0}+P_{0}$, which means  Put-Call parity is not satisfied under the Merton Model.

The put-call parity is $C_{0}= -Ke^{-rT}+S_{0}+P_{0}$,for Question 9 ,
As we can see
 $C_{0}= 6.81,-Ke^{-rT} = -78.91,S_{0} = 80,P_{0}$= 5.79.
So that $C_{0}\neq-Ke^{-rT}+S_{0}+P_{0}$, which means  Put-Call parity is not satisfied under the Merton Model.

**12. Run the Heston Model and Merton Model for 7 different strikes: 3 OTM calls; 1 ATM call; and 3 ITM calls. The strikes should be equally spaced. Try to use the following APPROXIMATE moneyness values: 0.85, 0.90, 0.95, 1, 1.05, 1.10, and 1.15. Recall that moneyness = stock/strike.**

In [ ]:
# @title
import numpy as np
import scipy.stats as ss

def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M  # T = maturity, M = number of time steps
    v = np.zeros((M + 1, Ite), dtype=np.float)
    v[0] = v0
    sdt = np.sqrt(dt)  # Sqrt of dt
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1]
            + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1, 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)

    return S

def random_number_gen(M, Ite):
    rand = np.random.standard_normal((2, M + 1, Ite))
    return rand

v0 = 0.032
kappa_v = 1.85
sigma_v = 0.35
theta_v = 0.045
rho = -0.30

S0 = 80  # Current underlying asset price
r = 0.055  # Risk-free rate
M0 = 50  # Number of time steps in a year
T = 3/12  # Number of years
M = int(M0 * T)  # Total time steps
Ite = 10000  # Number of simulations
dt = T / M  # Length of time step

# Generating random numbers from standard normal
rand = random_number_gen(M, Ite)


# Covariance Matrix
covariance_matrix = np.zeros((2, 2), dtype=np.float)
covariance_matrix[0] = [1.0, rho]
covariance_matrix[1] = [rho, 1.0]
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Volatility process paths
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)

# Underlying price process paths
S = Heston_paths(S0, r, V, 0, cho_matrix)

def heston_call_mc(S, K, r, T, t, type_of):
    if type_of == "C":
        payoff = np.maximum(0, S[-1, :] - K)
    else:
        payoff = np.maximum(0, K - S[-1, :])

    average = np.mean(payoff)

    return np.exp(-r * (T - t)) * average

<ipython-input-14-9853c0e40158>:52: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  covariance_matrix = np.zeros((2, 2), dtype=np.float)
<ipython-input-14-9853c0e40158>:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v = np.zeros((M + 1, Ite), dtype=np.float)


In [ ]:
# @title

# Define the list of moneyness values
moneyness_values = [0.85, 0.90, 0.95, 1.0, 1.05, 1.10, 1.15]

# Calculate strikes based on moneyness values
strikes = [S0 * m for m in moneyness_values]

# Initialize lists to store option prices
call_option_prices = []

# Calculate option prices for each strike
for strike in strikes:
    call_price = heston_call_mc(S, strike, r, T, 0, "C")
    call_option_prices.append(call_price)
    print(f"Strike Price: {strike}, Call Option Price: {call_price:.2f}")
for strike in strikes:
    put_price = heston_call_mc(S, strike, r, T, 0, "P")
    call_option_prices.append(put_price)
    print(f"Strike Price: {strike}, Put Option Price: {put_price:.2f}")


Strike Price: 68.0, Call Option Price: 12.08
Strike Price: 72.0, Call Option Price: 8.49
Strike Price: 76.0, Call Option Price: 5.34
Strike Price: 80.0, Call Option Price: 2.90
Strike Price: 84.0, Call Option Price: 1.33
Strike Price: 88.0, Call Option Price: 0.54
Strike Price: 92.0, Call Option Price: 0.20
Strike Price: 68.0, Put Option Price: 0.20
Strike Price: 72.0, Put Option Price: 0.55
Strike Price: 76.0, Put Option Price: 1.35
Strike Price: 80.0, Put Option Price: 2.85
Strike Price: 84.0, Put Option Price: 5.22
Strike Price: 88.0, Put Option Price: 8.38
Strike Price: 92.0, Put Option Price: 11.99


In [ ]:
# @title
from tabulate import tabulate

# Prepare your data as a list of lists
table_data = [
    ["12", "Call", "European", "Heston Model in combination with a correlation value of -0.30", "68","12.15"],
    ["12", "Call", "European", "Heston Model in combination with a correlation value of -0.30", "72","8.55"],
    ["12", "Call", "European", "Heston Model in combination with a correlation value of -0.30", "76","5.38"],
    ["12", "Call", "European", "Heston Model in combination with a correlation value of -0.30", "80","2.92"],
    ["12", "Call", "European", "Heston Model in combination with a correlation value of -0.30", "84","1.33"],
    ["12", "Call", "European", "Heston Model in combination with a correlation value of -0.30", "88","0.52"],
    ["12", "Call", "European", "Heston Model in combination with a correlation value of -0.30","92", "0.18"],
     ["12", "Put", "European", "Heston Model in combination with a correlation value of -0.30", "68","0.19"],
    ["12", "Put", "European", "Heston Model in combination with a correlation value of -0.30", "72","0.54"],
    ["12", "Put", "European", "Heston Model in combination with a correlation value of -0.30", "76","1.31"],
    ["12", "Put", "European", "Heston Model in combination with a correlation value of -0.30", "80","2.80"],
    ["12", "Put", "European", "Heston Model in combination with a correlation value of -0.30", "84","5.16"],
    ["12", "Put", "European", "Heston Model in combination with a correlation value of -0.30", "88","8.30"],
    ["12", "Put", "European", "Heston Model in combination with a correlation value of -0.30","92", "11.90"],
]

# Define the title for the table
table_title = "Summary of Results"

# Define the table headers
headers = ["Q #s", "Type", "Exer", "GWP 3 Method","Strikes" "Price"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

Summary of Results

+----+--------+----------+---------------------------------------------------------------+----------------+----------------+
|    | Q #s   | Type     | Exer                                                          |   GWP 3 Method |   StrikesPrice |
+====+========+==========+===============================================================+================+================+
| 12 | Call   | European | Heston Model in combination with a correlation value of -0.30 |             68 |          12.15 |
+----+--------+----------+---------------------------------------------------------------+----------------+----------------+
| 12 | Call   | European | Heston Model in combination with a correlation value of -0.30 |             72 |           8.55 |
+----+--------+----------+---------------------------------------------------------------+----------------+----------------+
| 12 | Call   | European | Heston Model in combination with a correlation value of -0.30 |             76

In [ ]:
# @title
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss
np.random.seed(0)



def merton_jump_paths(S0, r, sigma, T, lamb, muJ, deltaJ, n_paths, n_steps):
    dt = T / n_steps
    jump_intensity = lamb * (np.exp(muJ + 0.5 * deltaJ**2) - 1)

    paths = np.zeros((n_steps + 1, n_paths))
    paths[0] = S0

    for t in range(1, n_steps + 1):
        normal_shocks = np.random.normal(0, 1, n_paths)
        jump_shocks = np.random.normal(muJ, deltaJ, n_paths)
        jump_sizes = np.random.poisson(lamb * dt, n_paths)

        paths[t] = paths[t-1] * np.exp((r - 0.5 * sigma**2 - jump_intensity) * dt + sigma * np.sqrt(dt) * normal_shocks + jump_shocks * jump_sizes)

    return paths

def european_call_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps):
    paths = merton_jump_paths(S0, r, sigma, T, lamb, muJ, deltaJ, n_paths, n_steps)
    # Payoff at maturity for European call option
    payoff = np.maximum(paths[-1] - K, 0)
    # Discount the payoff to present value
    return np.mean(payoff) * np.exp(-r * T)

def european_put_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps):
    dt = T / n_steps
    discount = np.exp(-r * T)
    paths = merton_jump_paths(S0, r, sigma, T, lamb, muJ, deltaJ, n_paths, n_steps)

    # Payoff at maturity for a European put
    payoff = np.maximum(K - paths[-1], 0)

    return np.mean(payoff) * discount



european_call_price = european_call_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
print(f"European Call Option Price with jump intensity of 0.75: {european_call_price:.2f}")

european_put_price = european_put_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
print(f"European Put Option Price jump intensity of 0.75: {european_put_price:.2f}")

European Call Option Price with jump intensity of 0.75: 6.86
European Put Option Price jump intensity of 0.75: 5.76


In [ ]:
# @title
S0 = 80
r = 0.055
sigma = 0.35
T = 3 / 12
lamb = 0.75
muJ = -0.5
deltaJ = 0.22
n_paths = 100000
n_steps = 100
# Define the list of moneyness values
moneyness_values = [0.85, 0.90, 0.95, 1.0, 1.05, 1.10, 1.15]

# Calculate strikes based on moneyness values
strikes = [S0 * m for m in moneyness_values]

# Initialize lists to store option prices
call_option_prices = []

# Calculate option prices for each strike
for strike in strikes:
    european_call_price = european_call_merton(S0, strike, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
    call_option_prices.append(european_call_price)
    print(f"Strike Price: {strike}, Call Option Price: {european_call_price:.2f}")
for strike in strikes:
    european_put_price = european_put_merton(S0, strike, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
    call_option_prices.append(european_put_price)
    print(f"Strike Price: {strike}, Put Option Price: {european_put_price:.2f}")


Strike Price: 68.0, Call Option Price: 16.36
Strike Price: 72.0, Call Option Price: 13.33
Strike Price: 76.0, Call Option Price: 10.62
Strike Price: 80.0, Call Option Price: 8.29
Strike Price: 84.0, Call Option Price: 6.32
Strike Price: 88.0, Call Option Price: 4.66
Strike Price: 92.0, Call Option Price: 3.36
Strike Price: 68.0, Put Option Price: 3.36
Strike Price: 72.0, Put Option Price: 4.28
Strike Price: 76.0, Put Option Price: 5.68
Strike Price: 80.0, Put Option Price: 7.18
Strike Price: 84.0, Put Option Price: 9.11
Strike Price: 88.0, Put Option Price: 11.47
Strike Price: 92.0, Put Option Price: 14.13


In [ ]:
# @title
from tabulate import tabulate

# Prepare your data as a list of lists
table_data = [
    ["12", "Call", "European", "Merton Model in combination with jump intensity of 0.75", "68","16.29"],
    ["12", "Call", "European", "Merton Model in combination with jump intensity of 0.75", "72","13.40"],
    ["12", "Call", "European", "Merton Model in combination with jump intensity of 0.75", "76","10.63"],
    ["12", "Call", "European", "Merton Model in combination with jump intensity of 0.75", "80","8.24"],
    ["12", "Call", "European", "Merton Model in combination with jump intensity of 0.75", "84","6.33"],
    ["12", "Call", "European", "Merton Model in combination with jump intensity of 0.75", "88","4.69"],
    ["12", "Call", "European", "Merton Model in combination with jump intensity of 0.75", "92","3.38"],
    ["12", "Put", "European", "Merton Model in combination with jump intensity of 0.75", "68","3.40"],
    ["12", "Put", "European", "Merton Model in combination with jump intensity of 0.75", "72","4.36"],
    ["12", "Put", "European", "Merton Model in combination with jump intensity of 0.75", "76","5.56"],
    ["12", "Put", "European", "Merton Model in combination with jump intensity of 0.75", "80","7.22"],
    ["12", "Put", "European", "Merton Model in combination with jump intensity of 0.75", "84","9.14"],
    ["12", "Put", "European", "Merton Model in combination with jump intensity of 0.75", "88","11.55"],
    ["12", "Put", "European", "Merton Model in combination with jump intensity of 0.75", "92","14.17"],
]

# Define the title for the table
table_title = "Summary of Results"

# Define the table headers
headers = ["Q #s", "Type", "Exer", "GWP 3 Method","Strikes" "Price"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

Summary of Results

+----+--------+----------+---------------------------------------------------------+----------------+----------------+
|    | Q #s   | Type     | Exer                                                    |   GWP 3 Method |   StrikesPrice |
+====+========+==========+=========================================================+================+================+
| 12 | Call   | European | Merton Model in combination with jump intensity of 0.75 |             68 |          16.29 |
+----+--------+----------+---------------------------------------------------------+----------------+----------------+
| 12 | Call   | European | Merton Model in combination with jump intensity of 0.75 |             72 |          13.4  |
+----+--------+----------+---------------------------------------------------------+----------------+----------------+
| 12 | Call   | European | Merton Model in combination with jump intensity of 0.75 |             76 |          10.63 |
+----+--------+----------+--

**Step 2: All team members:**

**13. Repeat Questions 5 and 8 for the case of an American call option. Comment on the differences you observe from original Questions 5 and 8.**

In [ ]:
# @title
import numpy as np

def american_call_option(S, K, r, T, t):
    # Initialize the option value at maturity
    option_value = np.maximum(0, S[-1, :] - K)

    # Time step
    dt = T / len(S)

    # Perform backward induction
    for i in range(len(S) - 2, -1, -1):
        # Calculate the present value of option value
        option_value = np.exp(-r * dt) * (
            option_value  # Continue holding the option
        )

        # Calculate the intrinsic value (exercise value) of the option
        exercise_value = np.maximum(0, S[i, :] - K)

        # Update the option value if it's optimal to exercise
        option_value = np.maximum(option_value, exercise_value)

    # Calculate the average option value across all simulations
    option_price = np.mean(option_value)

    return option_price

# Usage
american_call_price = american_call_option(S, 80, 0.055, 3/12, 0)
print("American Call Price under Heston: ", american_call_price)


American Call Price under Heston:  4.61512675173979


In [ ]:
# @title
from tabulate import tabulate

# Prepare your data as a list of lists
table_data = [
    ["5", "Call", "European", "Heston Model in combination with Monte-Carlo simulation with a correlation value of -0.30", "2.83"],
    ["13", "Call", "American", "Heston Model in combination with Monte-Carlo simulation with a correlation value of -0.30", "3.62"]
]

# Define the title for the table
table_title = "Summary of Results"

# Define the table headers
headers = ["Q #s", "Type", "Exer", "GWP 3 Method", "Price"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

Summary of Results

+--------+--------+----------+-------------------------------------------------------------------------------------------+---------+
|   Q #s | Type   | Exer     | GWP 3 Method                                                                              |   Price |
+========+========+==========+===========================================================================================+=========+
|      5 | Call   | European | Heston Model in combination with Monte-Carlo simulation with a correlation value of -0.30 |    2.83 |
+--------+--------+----------+-------------------------------------------------------------------------------------------+---------+
|     13 | Call   | American | Heston Model in combination with Monte-Carlo simulation with a correlation value of -0.30 |    3.62 |
+--------+--------+----------+-------------------------------------------------------------------------------------------+---------+


**Comment on the differences you observe from original Questions 5**

To attain an American call option, we need to impelement the early exercise feature of American options. We do this by adding a backward induction loop to check whether it's optimal to exercise the option at each time step before maturity. Therefore, the price of an American call option is generally higher than the price of a European call option for several reasons, especially when using Monte Carlo simulations in the context of the Heston model. This is due to the early exercise option, optimal exercise timing as well as stochastic volatility and path-dependence. As regards early exercise optionality, American call options allow the holder to exercise the option at any time prior to or at expiration, but European options can only be executed at expiration. This is known as early exercise optionality. Because it enables the option holder to profit from positive price changes in the underlying asset, this early exercise provision gives American options an edge over other options. Due to this increased flexibility, American call options typically have higher prices (Marroni & Perdomo, 2014).

Additionally, when pricing an American option using simulations in the Heston model, the idea of optimal exercise time underlines the importance of timing at every level of simulation. This entails contrasting the calculated option value with the intrinsic value of the option, which is the difference between the current asset price and the strike price. It is profitable to exercise the American call option if the intrinsic value is greater than the option value. Comparing this feature to European options, which do not have this flexibility, may result in higher option prices (Marroni & Perdomo, 2014).

Lastly, on stochastic volatility and path-dependence, the Heston model introduces stochastic volatility, making the option pricing process more dynamic. American options benefit from this path-dependent nature, as the option holder can choose to exercise when volatility conditions are favorable, potentially increasing option values. European options, on the other hand, are path-independent and only depend on the asset price at expiration. The combination of stochastic volatility and path-dependence further contributes to the higher pricing of American call options in the Heston model (Marroni & Perdomo, 2014).


**For Team member B**

**Pricing American all using Mertons model**

In [ ]:
# @title
import numpy as np

def merton_jump_paths(S0, r, sigma, T, lamb, muJ, deltaJ, n_paths, n_steps):
    dt = T / n_steps
    jump_intensity = lamb * (np.exp(muJ + 0.5 * deltaJ**2) - 1)

    paths = np.zeros((n_steps + 1, n_paths))
    paths[0] = S0

    for t in range(1, n_steps + 1):
        normal_shocks = np.random.normal(0, 1, n_paths)
        jump_shocks = np.random.normal(muJ, deltaJ, n_paths)
        jump_sizes = np.random.poisson(lamb * dt, n_paths)

        paths[t] = paths[t-1] * np.exp((r - 0.5 * sigma**2 - jump_intensity) * dt + sigma * np.sqrt(dt) * normal_shocks + jump_shocks * jump_sizes)

    return paths

def american_option_lsm(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps):
    dt = T / n_steps
    discount = np.exp(-r * dt)
    paths = merton_jump_paths(S0, r, sigma, T, lamb, muJ, deltaJ, n_paths, n_steps)

    # Initialize payoffs at maturity
    payoff = np.maximum(paths[-1] - K, 0)

    for t in range(n_steps - 1, 0, -1):
        # Select in-the-money paths
        itm = paths[t] > K
        X = paths[t, itm]
        Y = payoff[itm] * discount

        # Using polynomial regression for continuation value estimation
        reg = np.polyfit(X, Y, 2)
        continuation = np.polyval(reg, X)

        # Update the payoff
        immediate_exercise = np.maximum(X - K, 0)
        payoff[itm] = np.where(immediate_exercise > continuation, immediate_exercise, payoff[itm] * discount)

    return np.mean(payoff) * discount


S0 = 80
K = 80
r = 0.055
sigma = 0.35
T = 3 / 12
lamb = 0.75
muJ = -0.5
deltaJ = 0.22
n_paths = 50000
n_steps = 100

american_call_price = american_option_lsm(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
print(f"American Call Option Price Using mertons model is : {american_call_price:.2f}")



American Call Option Price Using mertons model is : 8.36


**13) Comment on differences observed from original question 8**

- Sensitivity to Jump Parameters: The Merton model introduces jump parameters like the jump intensity (λ), mean (μ), and volatility (δ) of jumps. The value of American options, given their early exercise feature, will be more sensitive to these parameters compared to European options.
- Jump Impact on Early Exercise: The potential for large upward jumps (positive skewness) might decrease the incentive to exercise American calls early. If there's a significant chance of an advantageous price jump in the future,
-Computational Complexity: From a practical perspective, pricing American options under the Merton model is more computationally intensive due to the necessity to consider optimal stopping times at each time step. This complexity is due to the interaction between the early exercise feature and the jump dynamics, which doesn't exist in the European option scenario.

 **14. Using Heston model data from Question 6, price a European up-and-in call option (UAI) with a barrier level of \$95 and a strike price of \$95 as well. This UAI option becomes alive only if the stock price reaches (at some point before maturity) the barrier level (even if it ends below it). Compare the price obtained to the one from the simple European call.**

In [ ]:
# @title
import numpy as np

# Heston Model Parameters
v0 = 0.032
kappa_v = 1.85
theta_v = 0.045
sigma_v = 0.35
rho = -0.70  # Updated correlation value

# Option Parameters
S0 = 80
K = 95  # Strike Price
B = 95  # Barrier Level
r = 0.055  # Risk-free rate
T = 3/12  # Time to maturity (in years)
M0 = 50   # Number of time steps in a year
M = int(M0 * T)  # Total time steps
Ite = 10000  # Number of simulations
dt = T / M  # Length of time step

# Generate random numbers
rand = np.random.standard_normal((2, M + 1, Ite))

# Covariance Matrix
covariance_matrix = np.array([[1.0, rho], [rho, 1.0]])
cho_matrix = np.linalg.cholesky(covariance_matrix)

# Function to simulate volatility paths
def SDE_vol(v0, kappa, theta, sigma, T, M, Ite, rand, row, cho_matrix):
    dt = T / M
    v = np.zeros((M + 1, Ite), dtype=np.float)
    v[0] = v0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        v[t] = np.maximum(
            0,
            v[t - 1] + kappa * (theta - v[t - 1]) * dt
            + np.sqrt(v[t - 1]) * sigma * ran[row] * sdt,
        )
    return v

# Function to simulate stock price paths
def Heston_paths(S0, r, v, row, cho_matrix):
    S = np.zeros((M + 1, Ite), dtype=float)
    S[0] = S0
    sdt = np.sqrt(dt)
    for t in range(1, M + 1):
        ran = np.dot(cho_matrix, rand[:, t])
        S[t] = S[t - 1] * np.exp((r - 0.5 * v[t]) * dt + np.sqrt(v[t]) * ran[row] * sdt)
    return S

# Calculate option prices
V = SDE_vol(v0, kappa_v, theta_v, sigma_v, T, M, Ite, rand, 1, cho_matrix)
S = Heston_paths(S0, r, V, 0, cho_matrix)

# Initialize variables to track UAI call option price
uai_call_price = 0

# Iterate through simulation paths
for i in range(Ite):
    barrier_crossed = False  # Flag to check if barrier is crossed

    for t in range(1, M + 1):
        if S[t, i] >= B:
            barrier_crossed = True
            break  # Barrier crossed, exit inner loop

    if barrier_crossed:
        uai_call_price += np.maximum(0, S[-1, i] - K) * np.exp(-r * T)  # Calculate UAI call option price

# Calculate the average UAI call option price
uai_call_price /= Ite

# Print the UAI call option price
print("European Up-and-In Call Option Price:", uai_call_price)


European Up-and-In Call Option Price: 0.00735345336358276


<ipython-input-23-8c6a1f213008>:31: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  v = np.zeros((M + 1, Ite), dtype=np.float)


In [ ]:
# @title
from tabulate import tabulate

# Prepare your data as a list of lists
table_data = [
    ["5", "Call", "European", "ATM European Call for a simple Heston Model", "2.01"],
    ["5", "Call", "American", "Up-and-in Call option (UAI) with a barrier level/strike price $95", "0.01"]
]

# Define the title for the table
table_title = "Summary of Results"

# Define the table headers
headers = ["Q #s", "Type", "Exer", "GWP 3 Method", "Price"]

# Format the table using tabulate
table = tabulate(table_data, headers=headers, tablefmt="grid")

# Add the title above the table
table_with_title = f"{table_title}\n\n{table}"

# Print the formatted table
print(table_with_title)

Summary of Results

+--------+--------+----------+-------------------------------------------------------------------+---------+
|   Q #s | Type   | Exer     | GWP 3 Method                                                      |   Price |
+========+========+==========+===================================================================+=========+
|      5 | Call   | European | ATM European Call for a simple Heston Model                       |    2.01 |
+--------+--------+----------+-------------------------------------------------------------------+---------+
|      5 | Call   | American | Up-and-in Call option (UAI) with a barrier level/strike price $95 |    0.01 |
+--------+--------+----------+-------------------------------------------------------------------+---------+


**Compare the price obtained to the one from
the simple European call:**

Because the DAI put option requires additional requirements before it becomes active, its price is often lower than the price of a standard ATM European put option. These additional requirements are as follows (Marroni & Perdomo, 2014): (i) it is only activated (becomes alive) if the underlying stock price crosses or touches a predefined barrier level during the option's lifetime. If the stock price remains above the barrier level throughout the option's life, the DAI option expires worthless; (ii) the barrier level is typically set below the initial stock price, which means the stock price must move against the holder's favor to activate the option. These factors make DAI put options less likely to become active and pay out, which lowers their cost in comparison to a simple ATM European put option. The lower price reflects the lower probability of the DAI option being profitable (Marroni & Perdomo, 2014).

An ATM European put option, on the other hand, has no barrier requirements and pays out if the stock price is lower than the strike price at expiration. Because it has a larger chance of becoming profitable, it usually has a higher price. The price difference between these two options reflects the risk-reward trade-off associated with barrier options like the DAI put option. Investors are compensated for taking on the additional risk that the option may never become active, even if the stock price declines.


**15. Using Merton model data from Question 8, price a European down-and-in put option (DAI) with a barrier level of \$65 and a strike price of \$65 as well. This UAO option becomes alive only if the stock price reaches (at some point before maturity) the barrier level (even if it ends above it). Compare the price obtained to the one from the simple European put.**

In [ ]:
# @title
def european_down_and_in_put_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, barrier, n_paths, n_steps):
    dt = T / n_steps
    discount = np.exp(-r * T)
    paths = merton_jump_paths(S0, r, sigma, T, lamb, muJ, deltaJ, n_paths, n_steps)

    # Initialize an array to store the payoff for DAI option
    dai_payoffs = np.zeros(n_paths)

    for i in range(n_paths):
        hit_barrier = False  # Flag to check if the barrier is hit

        for t in range(1, n_steps + 1):
            if paths[t, i] <= barrier:
                hit_barrier = True
                break  # If barrier is hit, no need to continue checking

        # Calculate the payoff for DAI option
        if hit_barrier:
            dai_payoffs[i] = np.maximum(K - paths[-1, i], 0)

    return np.mean(dai_payoffs) * discount

barrier = 65
dai_option_price = european_down_and_in_put_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, barrier, n_paths, n_steps)
print(f"European Down-and-In Put Option Price with barrier at $65: {dai_option_price:.2f}")


European Down-and-In Put Option Price with barrier at $65: 9.96


In [ ]:
# @title
S0 = 80
K = 65
r = 0.055
sigma = 0.35
T = 3 / 12
lamb = 0.75
muJ = -0.5
deltaJ = 0.22
n_paths = 100000
n_steps = 100
european_put_price = european_put_merton(S0, K, r, T, sigma, lamb, muJ, deltaJ, n_paths, n_steps)
print(f"European Put Option Price with jump intensity of 0.75 at strike $65: {european_put_price:.2f}")


European Put Option Price with jump intensity of 0.75 at strike $65: 2.78


As we can see, the European Down-and-In Put Option Price with barrier is much larger than  the European Put Option Price with jump intensity of 0.75 at strike $65.

###REFERENCES
Schumacher, J. M. (2020). *Introduction to Financial Derivatives:
Modeling, Pricing and Hedging.* Open Press TiU: https://digi-courses.com/openpresstiu-introduction-to-financial-derivatives/


Marroni L. & I. Perdomo (2014). *Pricing and Hedging Financial Derivatives: A Guide for Practitioners.* Wiley